Authors: Antoine A. Ruzette
Date: 2025-02-21

This notebook processes cell measurement tables exported from QuPath to plot the spatial distribution of cell-level pixel intensity in relation to a modelled stromal border. It also supports the comparison of confusion matrices between threshold- and machine learning-based cell classification.

Contains the code to plot data from images containing four channels: DAPI (nuclei), TRITC (cytokeratin), FITC (fibronectin) and CY5 (Ki67). 

In [ ]:
! pip install "fitter>=1.6.0" "ipykernel>=6.29.5" "matplotlib>=3.10.0" "natsort>=8.4.0" "numpy>=2.2.3" "pandas>=2.2.3" "scipy>=1.15.2" "seaborn>=0.13.2" "setuptools>=75.8.0"

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fitter import Fitter
from scipy.stats import lognorm

# colorblind-friendly colors
CB_palette = ['#377eb8', '#ff7f00', '#4daf4a',
                '#f781bf', '#a65628', '#984ea3',
                '#999999', '#e41a1c', '#dede00']

## Back-end functions

In [2]:
def load_and_preprocess_files(folder_path, file_paths, expected_columns):
    """
    Load and preprocess CSV files.

    Args:
        folder_path (str): Path to the folder containing CSV files.
        file_paths (list): List of file names.
        expected_columns (dict): Dictionary mapping expected column keys to possible names.

    Returns:
        list: A list of processed DataFrames (one per image).
        dict: Column mapping for use in plotting.
    """
    dfs = []
    final_column_mapping = {}

    for image in file_paths:
        print(f"\n🔹 Processing: {image}")
        file_path = os.path.join(folder_path, image)

        # Load CSV to check available columns first
        try:
            df_sample = pd.read_csv(file_path, nrows=1)
            available_columns = df_sample.columns.tolist()
            print(f"✅ Available Columns: {available_columns}")
        except Exception as e:
            print(f"❌ Error loading {image}: {e}")
            continue

        # Dynamically map expected column names to available ones
        column_mapping = {}
        for key, possible_names in expected_columns.items():
            for name in possible_names:
                if name in available_columns:
                    column_mapping[key] = name
                    break
            else:
                print(f"⚠️ Warning: {key} column not found in {image}. Skipping.")

        if not column_mapping:
            print(f"⚠️ Skipping {image} as no expected columns were found.")
            continue

        # Reload dataframe with only found columns
        try:
            df = pd.read_csv(file_path, usecols=list(column_mapping.values()))
        except Exception as e:
            print(f"❌ Error loading selected columns in {image}: {e}")
            continue

        # Skip this file if essential columns are missing
        essential_columns = ["DAPI", "Ki67_647"]
        missing_essential = [col for col in essential_columns if col not in column_mapping]
        if missing_essential:
            print(f"⚠️ Skipping {image} due to missing essential columns: {missing_essential}")
            continue

        # Remove outliers dynamically (only for present columns)
        outlier_limits = {}
        for key in ["DAPI", "Ki67_647", "KER_488", "FN_568"]:
            if key in column_mapping:
                col_name = column_mapping[key]
                p01 = df[col_name].quantile(0.01)
                p99 = df[col_name].quantile(0.99)
                outlier_limits[col_name] = (p01, p99)

        print(f"📊 Outlier Thresholds: {outlier_limits}")

        # Filter outliers
        df_no_outlier = df.copy()
        for col, (p01, p99) in outlier_limits.items():
            df_no_outlier = df_no_outlier[(df_no_outlier[col] >= p01) & (df_no_outlier[col] <= p99)]

        if df_no_outlier.empty:
            print(f"⚠️ Skipping {image} as it became empty after outlier removal.")
            continue

        # ✅ Store actual filename instead of "1G", "2G" etc.
        df_no_outlier["Image"] = os.path.splitext(os.path.basename(image))[0]

        dfs.append(df_no_outlier)

        # Save column mapping for later use
        final_column_mapping = column_mapping  

    if dfs:
        return dfs, final_column_mapping
    else:
        print("⚠️ No valid data loaded.")
        return [], {}


## Exploring non-gaussian distributions using the `Fitter` library

### e.g., ki67 histograms

#### Finding the best fit distributions

In [ ]:
# Modify the following to use your data
########################################################

# Define folder and file paths
folder_path = "path/to/your/folder"

file_paths = [
    "data1.csv",
    "data2.csv",
]

# Expected column mappings
expected_columns = {
    "Class": ["Class"],
    "DAPI": ["DAPI: Nucleus: Median"],
    "KER_488": ["FITC KER: Cytoplasm: Median"],
    "Ki67_647": ["CY5 Ki67: Nucleus: Max"],
    "FN_568": ["TRITC FN: Cell: Median"],
    "Nucleus_Area": ["Nucleus: Area µm^2"]
}

# Define bin sizes for each channel
bin_sizes = {"DAPI": 150, "Ki67_647": 5, "KER_488": 50, "FN_568": 20}

########################################################

# Define folder and file paths
folder_path = "path/to/your/folder"

file_paths = [
    "data1.csv",
    "data2.csv",
]

# Define bin sizes for each channel
bin_sizes = {
    "DAPI": 50,
    "Ki67_647": 5,
    "KER_488": 75,
    "FN_568": 40
}

expected_columns = {
    "Class": ["Class"],
    "DAPI": ["DAPI: Nucleus: Median"],
    "KER_488": ["FITC KER: Cytoplasm: Median"],
    "Ki67_647": ["CY5 Ki67: Nucleus: Max"],
    "FN_568": ["TRITC FN: Cell: Median"],
    "Nucleus_Area": ["Nucleus: Area µm^2"]
}

# Define axis labels and limits
axis_labels = {
    "PDF": "Ki67 max\nnuclear intensity, a.u.",
    "CDF": "Ki67 max\nnuclear intensity, a.u."
}
axis_limits = {
    "PDF": (0, 3000),
    "CDF": (0, 3000)
}


# Load and preprocess data
dfs, column_mapping = load_and_preprocess_files(folder_path, file_paths, expected_columns)

# Initialize plots
fig, axs = plt.subplots(1, 2, figsize=(34, 16))

# Define bin width for histogram and fitting
bin_width = bin_sizes.get("Ki67_647", 5)

for idx, df in enumerate(dfs):
    image_name = df["Image"].iloc[0]
    gem_number = f"#{idx + 1}"

    # Get relevant column name
    pNDRG1_col = column_mapping.get("Ki67_647", "Ki67_647: Nucleus: Max")

    # Define bin edges based on min and max values
    min_val = df[pNDRG1_col].min()
    max_val = df[pNDRG1_col].max()
    bins = np.arange(min_val, max_val + bin_width, bin_width)

    # Fit log-normal distribution
    f = Fitter(df[pNDRG1_col], distributions=["lognorm"])
    f.fit()
    params = f.get_best(method="sumsquare_error").get("lognorm", {})

    # Compute fitted PDF and CDF at the same resolution as histogram
    pdf_values = lognorm.pdf(bins, *params.values())
    cdf_values = lognorm.cdf(bins, *params.values())
    pdf_values *= bin_width

    # Overlay histogram and best-fit PDF
    sns.histplot(df[pNDRG1_col], 
                 bins=bins, 
                 kde=False,
                 ax=axs[0], 
                 label=f'{gem_number}',
                 alpha=0.5,
                 stat='probability')
    axs[0].plot(bins, pdf_values, label=f"PDF {gem_number}", linewidth=7)

    # Plot CDF separately
    axs[1].plot(bins, cdf_values, label=f'CDF {gem_number}', linewidth=7)
    axs[1].set_ylim(0, 1)
# Apply formatting to match your layout
for key, ax in zip(["PDF", "CDF"], axs):
    ax.set_yscale('linear')
    ax.set_xscale('linear')
    ax.set_ylabel('Probability', fontsize=70)
    ax.set_xlabel(axis_labels[key], fontsize=70)
    ax.tick_params(axis='both', labelsize=55)

    # Set x-axis range and define 5 evenly spaced ticks
    if key in axis_limits:
        ax.set_xlim(axis_limits[key])
        xticks = np.linspace(axis_limits[key][0], axis_limits[key][1], 5)
        ax.set_xticks(xticks)

    # Ensure y-axis tick limits apply to all panels
    yticks = ax.get_yticks()
    if len(yticks) > 5:  
        min_y, max_y = min(yticks), max(yticks)  
        yticks = np.linspace(min_y, max_y, 5)  
        ax.set_yticks(yticks)

    # Format y-tick labels to 2 significant digits
    ax.set_yticklabels([f"{tick:.2g}" for tick in yticks])

plt.tight_layout()
plt.savefig("{folder_path}/best_fit_distribution.png", dpi=300)
plt.show()


#### Using the best fit distribution to translate a threshold

In [ ]:
# Modify the following to use your data
########################################################

# Define folder and file paths
folder_path = "path/to/your/folder"

file_paths = [
    "data1.csv",
    "data2.csv",
]

# Expected column mappings
expected_columns = {
    "Class": ["Class"],
    "DAPI": ["DAPI: Nucleus: Median"],
    "KER_488": ["FITC KER: Cytoplasm: Median"],
    "Ki67_647": ["CY5 Ki67: Nucleus: Max"],
    "FN_568": ["TRITC FN: Cell: Median"],
    "Nucleus_Area": ["Nucleus: Area µm^2"]
}

# Define bin sizes for each channel
bin_sizes = {"DAPI": 150, "Ki67_647": 5, "KER_488": 50, "FN_568": 20}

########################################################

# Load and preprocess data
dfs, column_mapping = load_and_preprocess_files(folder_path, file_paths, expected_columns)

# Initialize plots
fig, axs = plt.subplots(1, 2, figsize=(34, 15))

# Define bin width for histogram and fitting
bin_width = bin_sizes.get("Ki67_647", 5)

# Placeholder for storing reference distribution parameters
params_reference = None
threshold_reference = 950  # Example threshold value to translate

for idx, df in enumerate(dfs):
    image_name = df["Image"].iloc[0]
    gem_number = f"#{idx + 1}"
    # image_color = image_colors.get(image_name, "gray")
    pNDRG1_col = column_mapping.get("Ki67_647", "Ki67_647: Nuclear: Max")

    # Define bin edges based on min and max values
    min_val = df[pNDRG1_col].min()
    max_val = df[pNDRG1_col].max()
    bins = np.arange(min_val, max_val + bin_width, bin_width)

    # Fit log-normal distribution
    f = Fitter(df[pNDRG1_col], distributions=["lognorm"])
    f.fit()
    params = f.get_best(method="sumsquare_error").get("lognorm", {})

    if idx == 0:
        params_reference = params
    else:
        cdf_reference = lognorm.cdf(threshold_reference, **params_reference)
        translated_threshold = lognorm.ppf(cdf_reference, **params)
        scaled_threshold = (threshold_reference - params_reference['loc']) * (params['scale'] / params_reference['scale']) + params['loc']
        print(f"Translated threshold for img{idx}: {image_name}; {translated_threshold:.2f}")
        print(f"Scaled threshold for img{idx}: {scaled_threshold:.2f}")
        print("params_reference", params_reference)
        print("params", params)
        
    # Plot histograms and fitted distributions
    sns.histplot(df[pNDRG1_col], bins=bins, kde=False, ax=axs[0], alpha=0.5, stat='probability', label=gem_number)
    pdf_values = lognorm.pdf(bins, *params.values()) * bin_width
    cdf_values = lognorm.cdf(bins, *params.values())
    axs[0].plot(bins, pdf_values, label=f"PDF {gem_number}", linewidth=7)
    axs[1].plot(bins, cdf_values, label=f"CDF {gem_number}", linewidth=7)
    axs[1].set_ylim(0, 1)

for ax in axs:
    ax.legend(fontsize=20)
    ax.tick_params(axis='both', labelsize=20)
    ax.set_xlim(0, 8000)

plt.tight_layout()
# plt.savefig(os.path.join(folder_path, "fitted_measurements_distribution_pNDRG1.png"), dpi=300)
# plt.show()